In [ ]:
from datetime import date
from pathlib import Path

from loguru import logger
import sentier_data_tools as sdt

import os 

import brightway2 as bw
import lca_algebraic as agb
import numpy as np
from sympy import init_printing
import bw2io
import bw2data as bd
from dotenv import load_dotenv
import matplotlib.pyplot as plt
import pandas as pd
pd.set_option('display.max_rows', 500)
from sentier_data_tools import (
    Demand,
    Flow,
    FlowIRI,
    GeonamesIRI,
    ModelTermIRI,
    DatasetKind,
    ProductIRI,
    SentierModel,
)
# Pretty print for Sympy
init_printing()

In [ ]:
def create_pcb_local_datastorage(reset: bool = True):
    if reset:
        sdt.reset_local_database()

    df = pd.read_excel('Parameters_df.xlsx')
    assert len(COLUMNS) == len(UNITS)
    assert len(COLUMNS) == len(df.columns) -4

    metadata = sdt.Datapackage(
        name="pcb production data from EcoInvent3.10",
        description="inventory taken in EcoInvent3.10 database",
        contributors=[
            {
                "title": "Yannis Rosset",
                "path": "I don't have any website !",
                "role": "author",
            },
        ],
        homepage="https://ecoquery.ecoinvent.org/3.10/cutoff/dataset/9764/documentation",
    ).metadata()
    metadata.pop("version")

    for layers, finish, thick, prepreg in TYPES:
        filtered = df[(df["layers"] == layers) & (df["finish"] == finish) & (df["thickness"] == thick) & (df["prepregID"] == prepreg)].copy()
        logger.info("Adding {} records for type {}", len(filtered), (layers, finish, thick, prepreg))
        #filtered.drop(labels=["layers"], axis="columns", inplace=True)
        #filtered.columns = COLUMNS

        sdt.Dataset(
            name=f"pcb process model",
            dataframe=filtered,
            product=ProductIRI(
            "http://data.europa.eu/xsp/cn2024/853400000080"
        ),
            kind=DatasetKind.BOM,
            columns=[{"iri": x, "unit": y} for x, y in zip(COLUMNS, UNITS)],
            metadata=metadata,
            location="https://www.geonames.org/6295630/",
            version=1,
            valid_from=date(2018, 1, 1),
            valid_to=date(2028, 1, 1),
        ).save()


COLUMNS = [
    "http://data.europa.eu/xsp/cn2024/271600000080", #electricity
    "https://vocab.sentier.dev/product/energy/heat",  #heat ok 
    "https://vocab.sentier.dev/flows/envo/ENVO_00003065",  #ultrapure water ok 
    "https://vocab.sentier.dev/product/electronic_factory", 
    "http://data.europa.eu/xsp/cn2024/740300000080", #copper
    "http://data.europa.eu/xsp/cn2024/701913000080", #glass fiber
    "http://data.europa.eu/xsp/cn2024/390730000080", # Epoxy
    "http://data.europa.eu/xsp/cn2024/280600000080", #HCl
    "http://data.europa.eu/xsp/cn2024/284700000080", #H2O2
    "http://data.europa.eu/xsp/cn2024/282739200080", #Fe (III) Cl
    "http://data.europa.eu/xsp/cn2024/710800000080", #Au
    "http://data.europa.eu/xsp/cn2024/750610000080", #Ni
    "http://data.europa.eu/xsp/cn2024/710691000080", #Ag
    "http://data.europa.eu/xsp/cn2024/250100000080", #NaCl
    "http://data.europa.eu/xsp/cn2024/281512000080", #NaOH
    "http://data.europa.eu/xsp/cn2024/280700000080", #Sulfuric acid
    "http://data.europa.eu/xsp/cn2024/290919900080",  #Solvent
    "https://vocab.sentier.dev/flows/envo/ENVO_00003096",  # water ok

]

UNITS = [
    "https://vocab.sentier.dev/units/unit/KiloW-HR",
    "https://vocab.sentier.dev/units/unit/KiloW-HR",
    "https://vocab.sentier.dev/units/unit/L",
    "https://vocab.sentier.dev/units/unit/NUM", #part of a factory
    "https://vocab.sentier.dev/units/unit/KiloGM",
    "https://vocab.sentier.dev/units/unit/KiloGM",
    "https://vocab.sentier.dev/units/unit/KiloGM", #epoxy
    "https://vocab.sentier.dev/units/unit/L",
    "https://vocab.sentier.dev/units/unit/L",
    "https://vocab.sentier.dev/units/unit/L",
    "https://vocab.sentier.dev/units/unit/KiloGM",
    "https://vocab.sentier.dev/units/unit/KiloGM",
    "https://vocab.sentier.dev/units/unit/KiloGM",
    "https://vocab.sentier.dev/units/unit/L",
    "https://vocab.sentier.dev/units/unit/L",
    "https://vocab.sentier.dev/units/unit/L",
    "https://vocab.sentier.dev/units/unit/L",
    "https://vocab.sentier.dev/units/unit/L",
   
]
types = []
df = pd.read_excel('Parameters_df.xlsx')
for _, row in df.iterrows():
    types.append((row['layers'],row['finish'], row['thickness'],row['prepregID']))
TYPES = types


create_pcb_local_datastorage(True)

In [ ]:
class PCB_Model(SentierModel):
    provides = {
        ProductIRI(
            "http://data.europa.eu/xsp/cn2024/853400000080"
        ): "PCB",
    }
    aliases = {}

    
    def get_pcb_inventory(self):
        bom_pcb = self.get_model_data(
            ProductIRI(
            "http://data.europa.eu/xsp/cn2024/853400000080"
        ), kind=DatasetKind.BOM
        )
        return bom_pcb

    def run(self, layer, finish="Ni/Au", thickness=1.6, prepreg=1080) :
        self.layer = layer
        self.finish = finish
        self.thickness = thickness
        self.prepreg = prepreg
        bom_pcb= self.get_pcb_inventory()
        dfs= []
        for exatm in bom_pcb["exactMatch"]:
            if exatm.dataframe["layers"].values == self.layer and exatm.dataframe["finish"].values == self.finish and exatm.dataframe["thickness"].values == self.thickness and exatm.dataframe["prepregID"].values == self.prepreg:
                dfs.append(exatm.dataframe)
            
        fulldfs = pd.concat(dfs)
        fulldfs.drop(labels=["layers", "finish", "thickness", "prepregID"], axis="columns", inplace=True)
        fulldfs.rename(index = {0: 'amount'}, inplace = True)
        
        bom_pcb
        # filter or drop
        return fulldfs
        

In [189]:

demand= Demand(amount=1, unit_iri=sdt.UnitIRI("https://vocab.sentier.dev/units/unit/M2"),
               product_iri=sdt.ProductIRI("http://data.europa.eu/xsp/cn2024/853400000080"))

mm = PCB_Model(demand=demand, run_config=None)
mm.run(layer = 2, finish="Ag", thickness=1.6)

13:51:44 [info     ] Retrieved 50 triples from https://fuseki.d-d-s.ch/skosmos/query
13:51:44 [info     ] Retrieved 4 triples from https://fuseki.d-d-s.ch/skosmos/query


,Electricity,Energy,ultrapure water,Electronic component factory,CU input,GF input,Epoxy input,HCL,H2O2,Fe(III) Cl,Au,Ni,Ag,NaCl,NaOH,Sulfuric Acid,Solvent,Water
amount,35.9514,3.12621,98.3623,1e-12,0.281723,0.868,1.144,1.01998,0.144534,0.258215,0,0,0.001259,0.04621,1.53615,0.573233,0.0453,8.88739


In [ ]:
df_treatment = pd.read_excel('Parameters_df.xlsx')
df_treatment